<a href="https://colab.research.google.com/github/bogdanbabych/experiments_NLTK/blob/main/session_2_topic_modelling_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 2: Topic Modelling

In this session, we will focus on a popular kind of probabilistic Machine Learning algorithm. The algorithm itself is called 'latent dirichlet allocation' (LDA), but as it is the most popular algorithm used for 'topic modelling', it is often simply referred to as 'topic modelling'.

The aim of topic modelling is quite intuitive. We show the computer a selection of **documents**, and we ask it: What are these documents about? The computer examines the vocabulary of all the documents, and sorts the words into various **topics**. Now a human thinks of a 'topic' as a real-world thing or process which becomes a 'topic' of conversation. We all stand by a forest, point at it, and talk about how beautiful it is. The forest is the 'topic'. A computer cannot do this, so it approaches the question from another angle. It considers a 'topic' to be a set of words that tend to co-occur. When we talk about forests, we would tend to use the words 'tree', 'fox', 'path', 'dark', 'big', 'natural' and so on. We would not tend to use the words 'fricassé', 'printer' or 'transubstantiation'. When we use a computer to perform topic modelling, it sorts all the words in our corpus into clusters of co-occuring words. These clusters are the **topics**.

# The Algorithm

The LDA Algorithm can be summarised diagrammatically:

![LDA plate notation](https://upload.wikimedia.org/wikipedia/commons/4/4d/Smoothed_LDA.png)

$M$ denotes the number of documents

$N$ is number of words in a given document (document $i$ has $N_{i}$ words)

$\alpha$ is the parameter of the Dirichlet prior on the per-document topic distributions

$\beta$ is the parameter of the Dirichlet prior on the per-topic word distribution

$\theta_{i}$ is the topic distribution for document $i$

$\phi_{k}$ is the word distribution for topic $k$

$z_{ij}$ is the topic for the $j$-th word in document $i$

$w_{ij}$ is the specific word.

This diagram explains how the model works *generatively*. This is a generative model, because it learns how to create new documents based on the word/topic mixture of the corpus it is trained on. But topic moels are not generally actually *used* to generate text (partly because they have [no concept of word order](https://en.wikipedia.org/wiki/Bag-of-words_model)). Instead, once the topic model is trained, its internal parameters are examined to inform the user about the structure of the corpus, or the model is applied to the text to provide data for further analysis.

I step through the diagram in the [slides](slides/topic-modelling.pdf).

# Topic Modelling in Python

Now we have some grasp of what the algorithm is doing, we can learn to apply it in Python using the Gensim package.

If you want to do Topic Modelling in R, you can get very similiar results using [very similar code with the help of the MALLET or LDA packages](https://www.tidytextmining.com/topicmodeling.html).

## Data

For this tutorial we are going to use a small corpus of books from Project Gutenberg, which come included in the Natural Language Toolkit, a very useful text-analysis package for Python. Execute the cell below to import the NLTK and download the Gutenberg books.

In [1]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
gutenberg = nltk.corpus.gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
books = gutenberg.fileids()

print(f'Downloaded books:')
print("  - " + "\n  - ".join(books))

Downloaded books:
  - austen-emma.txt
  - austen-persuasion.txt
  - austen-sense.txt
  - bible-kjv.txt
  - blake-poems.txt
  - bryant-stories.txt
  - burgess-busterbrown.txt
  - carroll-alice.txt
  - chesterton-ball.txt
  - chesterton-brown.txt
  - chesterton-thursday.txt
  - edgeworth-parents.txt
  - melville-moby_dick.txt
  - milton-paradise.txt
  - shakespeare-caesar.txt
  - shakespeare-hamlet.txt
  - shakespeare-macbeth.txt
  - whitman-leaves.txt


## Preprocesing

We will be performing topic modelling using the Gensim library. It requires that the texts be turned into a 'bag of words' model first. A 'bag of words' model is a very simple model of texts, where each row is a *document*, and each column represents a particular *word*. Let's imagine that document 7 is *Moby Dick* and word 2223 is *whale*. In our big bag-of-words table, we would expect the number in row 7, column 2223 to be high, say $2000$. If document 64 is *Pride and Prejudice*, we would expect the number in column 2223 to be $0$, since whales are never mentioned in that novel.

If you read the literature in topic modelling, you will see it is common to chunk larger texts such as these into smaller units. However in this case the algorithm works quite well on the entire texts.

In [3]:
!pip install gensim
import gensim
from gensim.corpora import Dictionary # This will create the 'bag of words'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 44.3 MB/s eta 0:00:00


In [4]:



# Initialise the dictionary (this works out the vocab)
lower_case_corpus = [[word.lower() for word in gutenberg.words(book)] for book in books]
dictionary = Dictionary(lower_case_corpus)

# Filter out most and least common words
dictionary.filter_n_most_frequent(250)
dictionary.filter_extremes(no_below=5, no_above=0.99, keep_n=None)
dictionary.compactify()

# Create bag-of-words matrix
bag_of_words = [dictionary.doc2bow(doc) for doc in lower_case_corpus]

In [5]:
str(bag_of_words)

'[[(0, 144), (1, 16), (2, 6), (3, 338), (4, 2004), (5, 1), (6, 20), (7, 3), (8, 4), (9, 1), (10, 7), (11, 16), (12, 4), (13, 421), (14, 14), (15, 32), (16, 32), (17, 38), (18, 3), (19, 1382), (20, 85), (21, 14), (22, 1138), (23, 23), (24, 10), (25, 4), (26, 12), (27, 685), (28, 1), (29, 2), (30, 31), (31, 2), (32, 14), (33, 124), (34, 4), (35, 230), (36, 4), (37, 200), (38, 2), (39, 1), (40, 1), (41, 3), (42, 72), (43, 1), (44, 7), (45, 12), (46, 8), (47, 3), (48, 5), (49, 1), (50, 15), (51, 10), (52, 4), (53, 33), (54, 1), (55, 2), (56, 5), (57, 1), (58, 1), (59, 1), (60, 5), (61, 1), (62, 11), (63, 8), (64, 2), (65, 10), (66, 6), (67, 1), (68, 3), (69, 2), (70, 2), (71, 6), (72, 4), (73, 3), (74, 2), (75, 2), (76, 11), (77, 2), (78, 1), (79, 5), (80, 4), (81, 1), (82, 59), (83, 1), (84, 4), (85, 1), (86, 1), (87, 1), (88, 2), (89, 2), (90, 14), (91, 12), (92, 5), (93, 63), (94, 33), (95, 3), (96, 1), (97, 1), (98, 7), (99, 15), (100, 4), (101, 8), (102, 6), (103, 1), (104, 7), (105, 

## Training

Now that we have our corpus in the proper format, we can initialise and train a topic model on it. This will produce all the different things described in the diagram above: all the different probability distributions describing which topics are likely to appear where and which words are likely to appear in each topic.

In [6]:
from gensim.models import LdaModel

In [15]:


# We need to set some hyperparameters here
num_topics = 10
chunksize = 9 # There are 18 texts in the corpus
passes = 30
iterations = 400
eval_every = None

# Get mapping of word id numbers to the actual words out of the dictionary
# id2word = dictionary.id2token <-- this should work but is failing me
id2word = {val:key for key,val in dictionary.token2id.items()}
# del(topic_model)
# Now initialise and train the model (in Gensim you do this in one step, rather than defining the model then calling the .fit() method)
topic_model = LdaModel(
    corpus=bag_of_words,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

## Inference

Now we have trained the model, we can apply it to a particular text and see how it decomposes the text into topics. Execute the cell below to get the index number for each text in the corpus.

In [16]:
print(f'{"INDEX":5s} :: BOOK')
for idx, book in enumerate(books):
  print(f'{str(idx):5s} :: {book}')

INDEX :: BOOK
0     :: austen-emma.txt
1     :: austen-persuasion.txt
2     :: austen-sense.txt
3     :: bible-kjv.txt
4     :: blake-poems.txt
5     :: bryant-stories.txt
6     :: burgess-busterbrown.txt
7     :: carroll-alice.txt
8     :: chesterton-ball.txt
9     :: chesterton-brown.txt
10    :: chesterton-thursday.txt
11    :: edgeworth-parents.txt
12    :: melville-moby_dick.txt
13    :: milton-paradise.txt
14    :: shakespeare-caesar.txt
15    :: shakespeare-hamlet.txt
16    :: shakespeare-macbeth.txt
17    :: whitman-leaves.txt


In [18]:
topic_model.get_document_topics(bag_of_words[0])

[(0, np.float32(0.37044808)),
 (5, np.float32(0.5455588)),
 (7, np.float32(0.07483405))]

In [19]:
topic_model.show_topics(num_topics=-1, num_words=20)

[(0,
  '0.031*""" + 0.019*"have" + 0.017*"."" + 0.012*"mrs" + 0.012*"very" + 0.011*"--" + 0.011*"been" + 0.010*"every" + 0.009*"than" + 0.007*","" + 0.007*"only" + 0.007*"much" + 0.007*"own" + 0.007*".--" + 0.005*"herself" + 0.005*"sister" + 0.005*"miss" + 0.005*"!--" + 0.004*"other" + 0.004*"mother"'),
 (1,
  '0.000*"shall" + 0.000*"lord" + 0.000*"unto" + 0.000*"thou" + 0.000*",\'" + 0.000*"have" + 0.000*"god" + 0.000*"thee" + 0.000*"!\'" + 0.000*"ye" + 0.000*"king" + 0.000*"up" + 0.000*"4" + 0.000*"3" + 0.000*"5" + 0.000*"10" + 0.000*"hath" + 0.000*"people" + 0.000*"thy" + 0.000*"11"'),
 (2,
  '0.032*"shall" + 0.029*"unto" + 0.025*"lord" + 0.018*"thou" + 0.015*"thy" + 0.014*"god" + 0.013*"ye" + 0.013*"have" + 0.012*"thee" + 0.009*"1" + 0.009*"upon" + 0.009*"2" + 0.008*"israel" + 0.008*"king" + 0.008*"3" + 0.008*"4" + 0.008*"son" + 0.008*"up" + 0.008*"7" + 0.007*"5"'),
 (3,
  '0.000*"unto" + 0.000*"lord" + 0.000*"thou" + 0.000*"shall" + 0.000*"ye" + 0.000*"thee" + 0.000*"have" + 0.000

## Inspecting and Evaluating the Model

The model comes with numerous methods we can use to explore its structure. You can see all the methods that come with the model [in the official documentation](https://radimrehurek.com/gensim/models/ldamodel.html).

In [25]:
print(f'\nTop 3 Dominant topics for each book:')
for i, book_name in enumerate(books):
    # Get the topic distribution for the current document
    doc_topics = topic_model.get_document_topics(bag_of_words[i])

    # Sort topics by probability in descending order and take the top 3
    if doc_topics:
        sorted_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
        top_3_topics = sorted_topics[:3]

        print(f'{book_name:30s}:')
        for rank, (topic_id, topic_probability) in enumerate(top_3_topics):
            # Get the top words for the dominant topic for better interpretability
            top_words_for_topic = topic_model.show_topic(topic_id, topn=15)
            word_list = ', '.join([word for word, prob in top_words_for_topic])
            print(f'  Rank {rank + 1}: Topic {topic_id} (Prob: {topic_probability:.3f}): {word_list}')
    else:
        print(f'{book_name:30s} -> No topics found.')


Top 3 Dominant topics for each book:
austen-emma.txt               :
  Rank 1: Topic 5 (Prob: 0.546): ", very, mr, have, --, been, .", mrs, miss, anne, much, being, ,', never, quite
  Rank 2: Topic 0 (Prob: 0.370): ", have, .", mrs, very, --, been, every, than, ,", only, much, own, .--, herself
  Rank 3: Topic 7 (Prob: 0.075): ", ,", .", have, --, t, ?", upon, up, !", sir, father, very, never, oh
austen-persuasion.txt         :
  Rank 1: Topic 5 (Prob: 0.611): ", very, mr, have, --, been, .", mrs, miss, anne, much, being, ,', never, quite
  Rank 2: Topic 0 (Prob: 0.284): ", have, .", mrs, very, --, been, every, than, ,", only, much, own, .--, herself
  Rank 3: Topic 7 (Prob: 0.066): ", ,", .", have, --, t, ?", upon, up, !", sir, father, very, never, oh
austen-sense.txt              :
  Rank 1: Topic 0 (Prob: 0.845): ", have, .", mrs, very, --, been, every, than, ,", only, much, own, .--, herself
  Rank 2: Topic 7 (Prob: 0.134): ", ,", .", have, --, t, ?", upon, up, !", sir, father, ve

In [23]:
# To see the top words for a particular topic
# topic_model.show_topic()

# To see the top n most significant topics in the corpus
# topic_model.show_topics(num_topics=10, num_words=10)

# You should be able to calculate the 'log perplexity' using the below code
# This is useful if you want to compare the performance of two or more different
# models. Otherwise it it quite meaningless.
# topic_model.log_perplexity(bag_of_words)